# Homework 02. Chanwoo Kim (Student ID - 20185288)


In [3]:
# The function to read sequences from a FASTA file
def read_fasta(tmp_filename):
    rv = dict()
    f = open(tmp_filename, 'r')
    for line in f:
        if line.startswith('>'):
            tmp_h = line.strip().lstrip('>')
            rv[tmp_h] = ''
        else:
            rv[tmp_h] += line.strip().replace(' ', '')
    f.close()
    return rv

rc = {'A': 'T', 'T': 'A', 'G': 'C', 'C': 'G','S':'S','R':'Y'}
    # from the end of input sequence to the beginning, 
    # replace each nucleotide to its complementary one.
    #
    # The single line code at the end of this function is equivalent to the following 4 lines.
    # rv = []
    # for tmp_n in tmp_seq[::-1]:
    #     rv.append(rc[tmp_n])
    # return ''.join(rv)
def revcomp(tmp_seq):
     return ''.join([rc[x] for x in tmp_seq[::-1]])
    
# Read genomes
#Please choose a genome set and a primer set of your choice!
filename_genomes = '2019nCoV_genomes.2020_03_27.fa'
genome_list = read_fasta(filename_genomes)
#Read primers
filename_primers = '2019nCoV_primers.WHO.fa'
primer_list = read_fasta(filename_primers)

# This selects only the primer sets that are in the 'primer_list' for output of the program.
primer_type_database={'N1','N2','N3','RdRP_SARSr','E_Sarbeco','HKU-ORF1b-nsp','HKU-N','nCoV_IP2','nCoV_IP4','nCoV-ORF1ab','nCoV-N_China','RP-'}
primer_type=set()
for p_h in primer_list.keys():
    for l_h in primer_type_database:
        if l_h in p_h:
            primer_type.add(l_h)
primer_type=sorted(primer_type)

#Actual looping lart to generate the output.    
R_primer=''
F_primer=''
for p_type in primer_type: # loops the selected primer set one by one 
    del R_primer     # each loop, reverse and forward primers information are removed in the beginning
    del F_primer
    for p_h, p_seq in primer_list.items():
        if p_type in p_h and "F" in p_h.split()[0].lstrip(p_type): # for assigning reverse and forward primers for the selected primer set.
            F_primer=p_seq
            print('Forward primer is',p_h,F_primer)
        if p_type in p_h:
            if "R" in p_h.split()[0].lstrip(p_type) or 'RP-R'==p_h:
                R_primer=p_seq
                print('Reverse primer is',p_h,R_primer)
                R_primer=revcomp(R_primer)
    for g_h, g_seq in genome_list.items(): #actual output generation part
        if g_seq.find(F_primer)>1 and g_seq.find(R_primer,g_seq.find(F_primer))>1:
            print(g_h.split()[0], 'has the amplicon size:', 
                g_seq.find(R_primer)-g_seq.find(F_primer)+len(R_primer),'\nThe amplicon sequence :',  g_seq[g_seq.find(F_primer):g_seq.find(R_primer)]+R_primer) 
        elif 'R' or 'Y' in F_primer: # if primer contains any bases that are not 'A','T','G', and 'C'.
            F_primer_set=set()
            R_primer_set=set()
            Loop_no=0     #if this number is equal to zero, then  'does not contain the amplicon for this primer set' will be produced as output.      
            if 'R' or 'Y' in F_primer:
                F_primer_set.add(F_primer.replace('R','A').replace('Y','C'))#how to deal with the R,Y,S bases.
                F_primer_set.add(F_primer.replace('R','A').replace('Y','T'))
                F_primer_set.add(F_primer.replace('R','G').replace('Y','C'))
                F_primer_set.add(F_primer.replace('R','G').replace('Y','T'))
            if 'Y' or 'S' in R_primer:
                R_primer_set.add(R_primer.replace('Y','C').replace('S','C'))
                R_primer_set.add(R_primer.replace('Y','C').replace('S','G'))
                R_primer_set.add(R_primer.replace('Y','T').replace('S','C'))
                R_primer_set.add(R_primer.replace('Y','T').replace('S','G'))        
            for Fprimer in F_primer_set:
                for Rprimer in R_primer_set:
                    if g_seq.find(Fprimer)> -1 and g_seq.find(Rprimer, g_seq.find(Fprimer)) >-1:
                        print(g_h.split()[0], 'has the amplicon size:', g_seq.find(Rprimer)-g_seq.find(Fprimer)+len(Rprimer),
                              '\nThe amplicon sequence:',  g_seq[g_seq.find(Fprimer):g_seq.find(Rprimer)]+Rprimer)
                        Loop_no+=1
            if Loop_no==0:
                print(g_h.split()[0], 'does not contain the amplicon for this primer set')

            del F_primer_set
            del R_primer_set
    
        else:
            print(g_h.split()[0], 'does not contain the amplicon for this primer set2') #in case when the primer cannot detect any seq
   

Forward primer is E_Sarbeco_F1 Germany ACAGGTACGTTAATAGTTAATAGCGT
Reverse primer is E_Sarbeco_R2 Germany ATATTGCAGCAGTACGCACACA
MT246449 has the amplicon size: 113 
The amplicon sequence : ACAGGTACGTTAATAGTTAATAGCGTACTTCTTTTTCTTGCTTTCGTGGTATTCTTGCTAGTTACACTAGCCATCCTTACTGCGCTTCGATTGTGTGCGTACTGCTGCAATAT
MT246450 has the amplicon size: 113 
The amplicon sequence : ACAGGTACGTTAATAGTTAATAGCGTACTTCTTTTTCTTGCTTTCGTGGTATTCTTGCTAGTTACACTAGCCATCCTTACTGCGCTTCGATTGTGTGCGTACTGCTGCAATAT
MT246451 has the amplicon size: 113 
The amplicon sequence : ACAGGTACGTTAATAGTTAATAGCGTACTTCTTTTTCTTGCTTTCGTGGTATTCTTGCTAGTTACACTAGCCATCCTTACTGCGCTTCGATTGTGTGCGTACTGCTGCAATAT
MT246452 has the amplicon size: 113 
The amplicon sequence : ACAGGTACGTTAATAGTTAATAGCGTACTTCTTTTTCTTGCTTTCGTGGTATTCTTGCTAGTTACACTAGCCATCCTTACTGCGCTTCGATTGTGTGCGTACTGCTGCAATAT
MT246453 has the amplicon size: 113 
The amplicon sequence : ACAGGTACGTTAATAGTTAATAGCGTACTTCTTTTTCTTGCTTTCGTGGTATTCTTGCTAGTTACACTAGCCATCCTTACTGCGCTTCGATTGTGTGCGTACTGCTGCAAT

In [2]:
# The function to read sequences from a FASTA file
def read_fasta(tmp_filename):
    rv = dict()
    f = open(tmp_filename, 'r')
    for line in f:
        if line.startswith('>'):
            tmp_h = line.strip().lstrip('>')
            rv[tmp_h] = ''
        else:
            rv[tmp_h] += line.strip().replace(' ', '')
    f.close()
    return rv

rc = {'A': 'T', 'T': 'A', 'G': 'C', 'C': 'G','S':'S','R':'Y'}
    # from the end of input sequence to the beginning, 
    # replace each nucleotide to its complementary one.
    #
    # The single line code at the end of this function is equivalent to the following 4 lines.
    # rv = []
    # for tmp_n in tmp_seq[::-1]:
    #     rv.append(rc[tmp_n])
    # return ''.join(rv)
def revcomp(tmp_seq):
     return ''.join([rc[x] for x in tmp_seq[::-1]])
    
# Read genomes
#Please choose a genome set and a primer set of your choice!
filename_genomes = '2019nCoV_genomes.2020_03_27.fa'
genome_list = read_fasta(filename_genomes)
#Read primers
filename_primers = '2019nCoV_primers.fa'
primer_list = read_fasta(filename_primers)

# This selects only the primer sets that are in the 'primer_list' for output of the program.
primer_type_database={'N1','N2','N3','RdRP_SARSr','E_Sarbeco','HKU-ORF1b-nsp','HKU-N','nCoV_IP2','nCoV_IP4','nCoV-ORF1ab','nCoV-N_China','RP-'}
primer_type=set()
for p_h in primer_list.keys():
    for l_h in primer_type_database:
        if l_h in p_h:
            primer_type.add(l_h)
primer_type=sorted(primer_type)

#Actual looping lart to generate the output.    
R_primer=''
F_primer=''
for p_type in primer_type: # loops the selected primer set one by one 
    del R_primer     # each loop, reverse and forward primers information are removed in the beginning
    del F_primer
    for p_h, p_seq in primer_list.items():
        if p_type in p_h and "F" in p_h.split()[0].lstrip(p_type): # for assigning reverse and forward primers for the selected primer set.
            F_primer=p_seq
            print('Forward primer is',p_h,F_primer)
        if p_type in p_h:
            if "R" in p_h.split()[0].lstrip(p_type) or 'RP-R'==p_h:
                R_primer=p_seq
                print('Reverse primer is',p_h,R_primer)
                R_primer=revcomp(R_primer)
    for g_h, g_seq in genome_list.items(): #actual output generation part
        if g_seq.find(F_primer)>1 and g_seq.find(R_primer,g_seq.find(F_primer))>1:
            print(g_h.split()[0], 'has the amplicon size:', 
                g_seq.find(R_primer)-g_seq.find(F_primer)+len(R_primer),'\nThe amplicon sequence :',  g_seq[g_seq.find(F_primer):g_seq.find(R_primer)]+R_primer) 
        elif 'R' or 'Y' in F_primer: # if primer contains any bases that are not 'A','T','G', and 'C'.
            F_primer_set=set()
            R_primer_set=set()
            Loop_no=0     #if this number is equal to zero, then  'does not contain the amplicon for this primer set' will be produced as output.      
            if 'R' or 'Y' in F_primer:
                F_primer_set.add(F_primer.replace('R','A').replace('Y','C'))#how to deal with the R,Y,S bases.
                F_primer_set.add(F_primer.replace('R','A').replace('Y','T'))
                F_primer_set.add(F_primer.replace('R','G').replace('Y','C'))
                F_primer_set.add(F_primer.replace('R','G').replace('Y','T'))
            if 'Y' or 'S' in R_primer:
                R_primer_set.add(R_primer.replace('Y','C').replace('S','C'))
                R_primer_set.add(R_primer.replace('Y','C').replace('S','G'))
                R_primer_set.add(R_primer.replace('Y','T').replace('S','C'))
                R_primer_set.add(R_primer.replace('Y','T').replace('S','G'))        
            for Fprimer in F_primer_set:
                for Rprimer in R_primer_set:
                    if g_seq.find(Fprimer)> -1 and g_seq.find(Rprimer, g_seq.find(Fprimer)) >-1:
                        print(g_h.split()[0], 'has the amplicon size:', g_seq.find(Rprimer)-g_seq.find(Fprimer)+len(Rprimer),
                              '\nThe amplicon sequence:',  g_seq[g_seq.find(Fprimer):g_seq.find(Rprimer)]+Rprimer)
                        Loop_no+=1
            if Loop_no==0:
                print(g_h.split()[0], 'does not contain the amplicon for this primer set')

            del F_primer_set
            del R_primer_set
    
        else:
            print(g_h.split()[0], 'does not contain the amplicon for this primer set2') #in case when the primer cannot detect any seq
   

Forward primer is 2019-nCoV_N1-F GACCCCAAAATCAGCGAAAT
Reverse primer is 2019-nCoV_N1-R TCTGGTTACTGCCAGTTGAATCTG
MT246449 has the amplicon size: 72 
The amplicon sequence : GACCCCAAAATCAGCGAAATGCACCCCGCATTACGTTTGGTGGACCCTCAGATTCAACTGGCAGTAACCAGA
MT246450 has the amplicon size: 72 
The amplicon sequence : GACCCCAAAATCAGCGAAATGCACCCCGCATTACGTTTGGTGGACCCTCAGATTCAACTGGCAGTAACCAGA
MT246451 has the amplicon size: 72 
The amplicon sequence : GACCCCAAAATCAGCGAAATGCACCCCGCATTACGTTTGGTGGACCCTCAGATTCAACTGGCAGTAACCAGA
MT246452 has the amplicon size: 72 
The amplicon sequence : GACCCCAAAATCAGCGAAATGCACCCCGCATTACGTTTGGTGGACCCTCAGATTCAACTGGCAGTAACCAGA
MT246453 has the amplicon size: 72 
The amplicon sequence : GACCCCAAAATCAGCGAAATGCACCCCGCATTACGTTTGGTGGACCCTCAGATTCAACTGGCAGTAACCAGA
MT246454 has the amplicon size: 72 
The amplicon sequence : GACCCCAAAATCAGCGAAATGCACCCCGCATTACGTTTGGTGGACCCTCAGATTCAACTGGCAGTAACCAGA
MT246455 has the amplicon size: 72 
The amplicon sequence : GACCCCAAAATCAGCGAAATGCACCCCGCA